## Create database

In [13]:
from pymilvus import connections, db

conn = connections.connect(host='10.37.239.102', port=19530)

In [15]:
# database = db.create_database("face_recognition")

db.using_database("face_recognition")

In [16]:
db.list_database()

['face_recognition', 'default']

## Create collection

In [7]:
from pymilvus import CollectionSchema, FieldSchema, DataType

face_id = FieldSchema(name="face_id", description="id", dtype=DataType.INT64, is_primary=True, auto_id=False)
face_vector = FieldSchema(name="face_vector", description="face", dtype=DataType.FLOAT_VECTOR, dim=512)
schema = CollectionSchema(fields=[face_id, face_vector], description="face recognition", enable_dynamic_field=True)
collection_name = "face_recognition"

In [8]:
from pymilvus import Collection
collection = Collection(
    name=collection_name,
    schema=schema,
    )

In [9]:
from pymilvus import utility
utility.list_collections()

['hello_milvus', 'face_recognition']

## Insert entities

In [26]:
import numpy as np

arr_id = []
arr_face = []
for i in range(29):
    data = np.load("deep_sort/emb_db/{}.npy".format(i))
    for d in data:
        arr_id.append(i)
        arr_face.append(d)
entities = [
    [i for i in arr_id],
    [d.tolist() for d in arr_face]
]

# collection.insert(entities)
# collection.flush()

## Builds indexes

In [10]:
index_params = {
    "metric_type":"L2", 
    "index_type":"IVF_FLAT",
    "params":{"nlist":512}
}

collection.create_index(
    field_name="face_vector",
    index_params=index_params
)

utility.index_building_progress("face_recognition")

{'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0}

## Search

In [20]:
from pymilvus import Collection
collection = Collection("face_recognition")      # Get an existing collection.
collection.load()


In [35]:
vectors_to_search = entities[-1][-2:]
print(vectors_to_search)
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

[[0.17638534307479858, 0.42017826437950134, 1.4904439449310303, 0.07388012111186981, -0.2290116399526596, 0.38223323225975037, 0.14843469858169556, -1.6052991151809692, -0.4905577600002289, 0.07135271281003952, 0.4755544066429138, -0.4165877401828766, 0.0642831027507782, 0.4728319048881531, -0.5804842114448547, -0.7982694506645203, 0.4621158242225647, -0.09434220939874649, 0.6876658201217651, 0.9743063449859619, 0.20437628030776978, -0.5370941162109375, -0.014810314401984215, -0.47231435775756836, -0.044075436890125275, -0.004678304772824049, 0.5796526670455933, 0.6026009917259216, -0.5528243184089661, -0.7814216017723083, 0.24128149449825287, 1.061115026473999, -0.19300174713134766, 0.5500689744949341, -0.2442205250263214, -0.3916594982147217, -0.046508822590112686, -1.062690258026123, 0.47030195593833923, -0.42249011993408203, -0.11664146184921265, 0.11852696537971497, -0.19977949559688568, -0.01880726031959057, 0.3772212862968445, 1.0054353475570679, -0.10584604740142822, -0.7117444

In [33]:
# collection.load()
res = collection.query(
    expr="face_id in [0]",
    offset=0,
    limit=3,
    output_fields=["face_id", "face_vector"]
)
print(res)

[{'face_vector': [-0.17053774, 0.37509125, -0.42335105, 0.19297464, -0.0051200353, -0.42582282, 0.9842017, 0.50458723, 0.09686308, 0.57177055, -1.094933, 0.4809867, -0.030772785, 0.8096027, -0.15230195, -0.24945253, -0.501412, 0.41793162, 0.17472507, -0.7619938, 0.3229389, -0.5992483, -1.0581375, 0.47509256, 0.8019096, 0.4336139, -0.36763066, -0.16005939, -0.06495239, 0.25445595, -0.45762625, -0.7644256, 0.6779863, -0.71104777, 1.6012626, 0.018900543, 0.37187418, -0.19320063, -0.21771474, -0.73876697, -0.051138207, 0.45440114, 0.54619306, 0.81244814, 0.06084915, 0.9873841, -0.29309678, -0.8074995, -0.2647026, 1.2462305, 1.0083389, -1.3193908, -0.5297571, 0.10229367, 0.08567942, 0.15281914, 0.22152677, 0.03904917, 0.19527513, 0.12739366, -0.46672705, -0.34366983, 0.7400071, -0.075910755, -0.19415663, -0.48141778, -0.11105879, 0.18555614, 0.35343713, 0.35908842, 1.1761405, 0.1950299, -0.30111524, -0.3565288, -0.44610807, -0.6512591, 0.55538404, -0.19417505, 1.1908232, 0.62830156, -0.5006

In [36]:
collection.load()
collection.search(vectors_to_search, "face_vector", search_params, limit=1, output_fields=["face_id"])

['["id: 28, distance: 0.0, entity: {\'face_id\': 28}"]', '["id: 28, distance: 0.0, entity: {\'face_id\': 28}"]']

## Drop database

In [ ]:
from pymilvus import utility, db

db.using_database("face_recognition")
utility.drop_collection("face_recognition")
db.drop_database("face_recognition")

db.list_database()

# TESTING


In [17]:
from pymilvus import Collection
collection = Collection('face_recognition')

print(collection.num_entities)

1535


In [18]:
from pymilvus import utility
utility.list_collections()


['face_recognition']